# Analysis on cleaned dataset from Data.ipynb

In [112]:
#only perform once in the beginning
#%pip install linearmodels

In [187]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'


# Fixed effects regression:
from linearmodels import PanelOLS # for FE
import statsmodels.api as sm # for OLS


In [222]:
# import the csv data from Data.ipynb
data = pd.read_csv('/Users/maxweber/Desktop/DataMasterThesis/data_clean.csv')
houshold = pd.read_csv('/Users/maxweber/Desktop/DataMasterThesis/houshold_clean.csv', index_col=['SSUID', 'month_total'])

In [10]:
def get_grouping(df_column, group_dict, reb_dummies_df, naming_interaction = 'j'):
    # get flag indicating the group with the dictionary-mapping:
    new_col = df_column.apply(lambda x: next((k for k, v in group_dict.items() if x in v), 0))
    new_col = new_col.rename(new_col.name + '_flag')
    new_cols = pd.get_dummies(new_col) # get dummies from group flag


   # create interaction dummies (group-dummy * rebatelag_dummy) iterating over each group_id and concatenating in the end:
    df_group_interactions = pd.DataFrame(df_column) # for initialization with correct index
    group_lst = group_dict.keys()
    for group_id in group_lst:
        dummies_group = reb_dummies_df.multiply(new_cols[group_id], axis = 'index')
        
        new_dummy_names = []
        for name in dummies_group.columns:
            new_dummy_names.append(name + '-' + naming_interaction + str(group_id))
        dummies_group.columns = new_dummy_names
        df_group_interactions = df_group_interactions.merge(dummies_group,left_index= True, right_index=True)
        df_group_interactions.replace(np.NaN, 0, inplace = True)
    return df_group_interactions

# Base Setup for FE 2SLS Estimation of the Average Treatment Effect

Approach is as in Powell (2020) Table 6. Without subsetting/grouping. He reported a similar estimation in the online Appendix A.3 (First stage and mean estimates).

In [144]:
# this is the dataframe grouped by HH
houshold = pd.read_csv('/Users/maxweber/Desktop/DataMasterThesis/houshold_clean.csv', index_col=['SSUID', 'month_total'])

houshold.reset_index(drop=False, inplace=True)
houshold.fillna(0,inplace=True)
houshold.columns

Index(['SSUID', 'month_total', 'tfearn', 'erbamth', 'TFTOTINC', 'ERBATAMT',
       'ERBAMTH', 'SREFMON', 'ems', 'EHHNUMPP', 'EREBATOC', 'erebate-6',
       'erebate-5', 'erebate-4', 'erebate-3', 'erebate-2', 'erebate-1',
       'erebate0', 'erebate1', 'erebate2', 'erebate3', 'erebate4', 'erebate5',
       'erebate6', 'erebate7', 'erebate8', 'erebate9', 'erebate10',
       'erebate11'],
      dtype='object')

In [145]:
# analysis on subset with time-interactions terms:
hh_new = houshold[['ERBATAMT', 'ERBAMTH', 'tfearn', 'month_total', 'SSUID','ems', 'EHHNUMPP', 'SREFMON', 'EREBATOC']]
hh_new['reb'] = (houshold['erebate0'] + houshold['erebate1']).astype(int)
hh_new['reb_lag'] = (houshold['erebate2'] + houshold['erebate3']).astype(int)
hh_new['erbatamt'] = hh_new['ERBATAMT'].multiply(hh_new['reb'], axis = 'index')  # so that it is 0 in the months without rebate payout
hh_new['erbatamt_lag'] = hh_new['ERBATAMT'].multiply(hh_new['reb_lag'], axis = 'index')

hh_new.set_index(['SSUID', 'month_total'], drop=False, inplace=True)

#interaction terms based on  marital status, household size, month (and month relative to interview month)
hh_new['time_interact'] = 'hh-'+ hh_new['month_total'].astype(int).astype(str) + '-' +  hh_new['EHHNUMPP'].astype(str) + '-'  + hh_new['ems'].astype(str) 
print('# unique: ', len(hh_new['time_interact'].unique()))

# define the time-interaction terms as own dataframe
time_interact = pd.DataFrame(hh_new['time_interact'], index = hh_new.index)
time_interact.head()

# unique:  121


time_interact
SSUID       month_total              
19128000276 6.0              hh-6-3-1
            7.0              hh-7-3-1
            8.0              hh-8-3-1
            9.0              hh-9-3-1
            10.0            hh-10-3-1

# 1st Stage FE

In [146]:
# first stage to estimate fitted rebate with interactions with time effect
exog_vars = hh_new[['reb', 'reb_lag']]
#time_interact = hh_new['time_interact'].to_frame()
exog = sm.add_constant(exog_vars)

model_S1_erbatamt = PanelOLS(hh_new['erbatamt'], exog, entity_effects = True, time_effects = False, other_effects = time_interact, check_rank = True, drop_absorbed=True)
params_S1_erbatamt = model_S1_erbatamt.fit() #cov_type='clustered', cluster_entity = True
params_S1_erbatamt

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,erbatamt,R-squared:,0.7296
Estimator:,PanelOLS,R-squared (Between):,0.6356
No. Observations:,196864,R-squared (Within):,0.7677
Date:,"Tue, Jun 14 2022",R-squared (Overall):,0.7467
Time:,16:00:56,Log-likelihood,-1.266e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.322e+05
Entities:,24608,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(2,172134)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,2.322e+05


In [147]:
# first stage to estimate fitted rebate-lag with interactions with time effect
exog_vars = hh_new[['reb', 'reb_lag']]
#time_interact = hh_new['time_interact'].to_frame()
exog = sm.add_constant(exog_vars)

model_S1_erbatamt = PanelOLS(hh_new['erbatamt_lag'], exog, entity_effects = True, time_effects = False, other_effects = time_interact, check_rank = True, drop_absorbed=True)
params_S1_erbatamt_lag = model_S1_erbatamt.fit() #cov_type='clustered', cluster_entity = True
params_S1_erbatamt_lag

Dep. Variable:,erbatamt_lag,R-squared:,0.7203
Estimator:,PanelOLS,R-squared (Between):,0.5425
No. Observations:,196864,R-squared (Within):,0.7571
Date:,"Tue, Jun 14 2022",R-squared (Overall):,0.7246
Time:,16:01:33,Log-likelihood,-1.311e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.216e+05
Entities:,24608,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(2,172134)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,2.216e+05


# 1st Stage OLS

In [206]:
hh_ols = hh_new
hh_ols.reset_index(drop=True, inplace = True)
hh_ols.set_index('SSUID')
hh_ols.columns

Index(['ERBATAMT', 'ERBAMTH', 'tfearn', 'month_total', 'SSUID', 'ems',
       'EHHNUMPP', 'SREFMON', 'EREBATOC', 'reb', 'reb_lag', 'erbatamt',
       'erbatamt_lag', 'time_interact', 'erbatamt_fitted_base_reb',
       'erbatamt_fitted_base_reb_lag', 'const'],
      dtype='object')

In [208]:
y = hh_ols['erbatamt']
hh_ols['const'] = 1
dummies_ols = pd.get_dummies(hh_ols[[ 'ems', 'EHHNUMPP', 'SREFMON', 'month_total']])

X = hh_ols[['const', 'reb', 'reb_lag']].merge(dummies_ols, left_index=True, right_index=True)

In [209]:
model_ols_reb = sm.OLS(y,X)
model_ols_reb.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               erbatamt   R-squared:                       0.758
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                 3.618e+04
Date:                Tue, 14 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:40:10   Log-Likelihood:            -1.2974e+06
No. Observations:              196864   AIC:                         2.595e+06
Df Residuals:                  196846   BIC:                         2.595e+06
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             4.494e+11   2.26e+12      0.199      0.842   -3.98e+12    4.88e+12
reb                968.5658      1.744    555.369      0.000     965.148     971.984
reb_lag              0.6744      1.321      0.511      0.610      -1.914       3.263
EHHNUMPP            18.3074      0.350     52.298      0.000      17.621      18.993
SREFMON              0.5231      0.422      1.240      0.215      -0.304       1.350
ems_0             2.829e+11   8.36e+11      0.338      0.735   -1.36e+12    1.92e+12
ems_1             2.829e+11   8.36e+11      0.338      0.735   -1.36e+12    1.92e+12
month_total_10.0 -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_11.0 -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_12.0 -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_13.0 -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_14.0 -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_15.0 -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_5.0  -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_6.0  -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_7.0  -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_8.0  -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
month_total_9.0  -7.323e+11   2.73e+12     -0.269      0.788   -6.08e+12    4.61e+12
==============================================================================
Omnibus:                   145186.858   Durbin-Watson:                   1.250
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         12849594.268
Skew:                           2.859   Prob(JB):                         0.00
Kurtosis:                      42.164   Cond. No.                     9.78e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.56e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# 2nd Stage and Fitted Values

In [148]:
# adding fitted values to dataframe in preparation of 2nd stage regression:
print(params_S1_erbatamt.params.loc[['reb', 'reb_lag']])
print(params_S1_erbatamt_lag.params.loc[['reb', 'reb_lag']])


erbatamt_fitted_base_reb = params_S1_erbatamt.fitted_values
erbatamt_fitted_base_reb.columns = ['erbatamt_fitted_base_reb']

erbatamt_fitted_base_reb_lag = params_S1_erbatamt_lag.fitted_values
erbatamt_fitted_base_reb_lag.columns = ['erbatamt_fitted_base_reb_lag']

hh_new = pd.concat([hh_new,erbatamt_fitted_base_reb, erbatamt_fitted_base_reb_lag], axis = 1)

reb        964.382407
reb_lag     -3.491786
Name: parameter, dtype: float64
reb         -7.544792
reb_lag    971.146627
Name: parameter, dtype: float64


In [149]:
# 2nd stage FE regression:
exog_vars = hh_new[['erbatamt_fitted_base_reb', 'erbatamt_fitted_base_reb_lag']]
exog = sm.add_constant(exog_vars)
exog
model_second_stage = PanelOLS(hh_new['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_second_stage = model_second_stage.fit()
params_second_stage

Dep. Variable:,tfearn,R-squared:,6.194e-06
Estimator:,PanelOLS,R-squared (Between):,4.711e-05
No. Observations:,196864,R-squared (Within):,-3.745e-05
Date:,"Tue, Jun 14 2022",R-squared (Overall):,3.702e-05
Time:,16:01:34,Log-likelihood,-1.776e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.5335
Entities:,24608,P-value,0.5866
Avg Obs:,8.0000,Distribution:,"F(2,172244)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,0.5335


In [159]:
print(hh_new['erbatamt_fitted_base_reb'].unique())
print(hh_new['erbatamt_fitted_base_reb_lag'].unique())

print(hh_new[['erbatamt_fitted_base_reb', 'erbatamt_fitted_base_reb_lag']].describe())
hh_new[hh_new['erbatamt'] > 0] [['erbatamt_fitted_base_reb', 'erbatamt_fitted_base_reb_lag']].describe() 


[  1.68944599 966.0718533   -1.80233988]
[  2.47889537  -5.06589683 973.62552284]
       erbatamt_fitted_base_reb  erbatamt_fitted_base_reb_lag
count             196864.000000                 196864.000000
mean                 110.682999                    168.690243
std                  306.290303                    366.918514
min                   -1.802340                     -5.065897
25%                    1.689446                      2.478895
50%                    1.689446                      2.478895
75%                    1.689446                      2.478895
max                  966.071853                    973.625523


,erbatamt_fitted_base_reb,erbatamt_fitted_base_reb_lag
count,2.237200e+04,2.237200e+04
mean,9.660719e+02,-5.065897e+00
std,1.148263e-11,1.591651e-12
min,9.660719e+02,-5.065897e+00
25%,9.660719e+02,-5.065897e+00
50%,9.660719e+02,-5.065897e+00
75%,9.660719e+02,-5.065897e+00
max,9.660719e+02,-5.065897e+00


# Heterogenous Treatment effect estimation as in Parker (2014):

1. ATE estimation

2. HTE in married/unmmaried HHs

3. HTE in familysize


## Average Treatment Effect

In [232]:
hh_gr1 = hh_new
hh_gr3['month_total'] = hh_gr3['month_total'].astype(float).astype(int)
hh_gr3.set_index(['SSUID', 'month_total'], inplace=True, drop=False)
hh_gr1.head()

ERBATAMT  ERBAMTH  tfearn  month_total        SSUID  \
SSUID       month_total                                                        
19128000276 6                 0.0     -1.0       0            6  19128000276   
            7                 0.0     -1.0       0            7  19128000276   
            8                 0.0     -1.0       0            8  19128000276   
            9                 0.0     -1.0       0            9  19128000276   
            10                0.0     -1.0       0           10  19128000276   

                        ems  EHHNUMPP  SREFMON  EREBATOC  reb  reb_lag  \
SSUID       month_total                                                  
19128000276 6             1         3        1      -1.0    0        0   
            7             1         3        2      -1.0    0        0   
            8             1         3        3      -1.0    0        0   
            9             1         3        4      -1.0    0        0   
            10            1         3        1      -1.0    0        0   

                         erbatamt  erbatamt_lag time_interact  \
SSUID       month_total                                         
19128000276 6                 0.0           0.0      hh-6-3-1   
            7                 0.0           0.0      hh-7-3-1   
            8                 0.0           0.0      hh-8-3-1   
            9                 0.0           0.0      hh-9-3-1   
            10                0.0           0.0     hh-10-3-1   

                         erbatamt_fitted_base_reb  \
SSUID       month_total                             
19128000276 6                            1.689446   
            7                            1.689446   
            8                            1.689446   
            9                            1.689446   
            10                           1.689446   

                         erbatamt_fitted_base_reb_lag  const  reb_avg_amt  \
SSUID       month_total                                                     
19128000276 6                                2.478895      1          0.0   
            7                                2.478895      1          0.0   
            8                                2.478895      1          0.0   
            9                                2.478895      1          0.0   
            10                               2.478895      1          0.0   

                         reb_lag_avg_amt  
SSUID       month_total                   
19128000276 6                        0.0  
            7                        0.0  
            8                        0.0  
            9                        0.0  
            10                       0.0

In [227]:
reb_amt_avg = round(hh_gr1[hh_gr1['ERBATAMT']>0]['ERBATAMT'].mean(),2) # 956.60
reb_amt_avg

956.6

In [228]:
# this is only needed with different groups!
#hh_gr1['erbatamt'] = hh_gr1['ERBATAMT'].multiply(hh_gr1['reb'], axis = 'index')  # so that it is 0 in the months without rebate payout
#hh_gr1['erbatamt_lag'] = hh_gr1['ERBATAMT'].multiply(hh_gr1['reb_lag'], axis = 'index')

hh_gr1['reb_avg_amt'] = hh_gr1['reb']*reb_amt_avg
hh_gr1['reb_lag_avg_amt'] = hh_gr1['reb_lag']*reb_amt_avg

In [233]:
exog_vars = hh_gr1[['reb_avg_amt', 'reb_lag_avg_amt']]
exog = sm.add_constant(exog_vars)
exog
model_ate = PanelOLS(hh_gr1['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_ate = model_ate.fit()
params_ate

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,tfearn,R-squared:,6.194e-06
Estimator:,PanelOLS,R-squared (Between):,4.711e-05
No. Observations:,196864,R-squared (Within):,-3.745e-05
Date:,"Thu, Jun 16 2022",R-squared (Overall):,3.702e-05
Time:,10:42:30,Log-likelihood,-1.776e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.5335
Entities:,24608,P-value,0.5866
Avg Obs:,8.0000,Distribution:,"F(2,172244)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,0.5335


## Grouping 1: Marriage

In [234]:
hh_gr2 = hh_new
hh_gr2['month_total'] = hh_gr2['month_total'].astype(float).astype(int)
hh_gr2.set_index(['SSUID', 'month_total'], inplace=True, drop=False)
hh_gr2.head()

ERBATAMT  ERBAMTH  tfearn  month_total        SSUID  \
SSUID       month_total                                                        
19128000276 6                 0.0     -1.0       0            6  19128000276   
            7                 0.0     -1.0       0            7  19128000276   
            8                 0.0     -1.0       0            8  19128000276   
            9                 0.0     -1.0       0            9  19128000276   
            10                0.0     -1.0       0           10  19128000276   

                        ems  EHHNUMPP  SREFMON  EREBATOC  reb  reb_lag  \
SSUID       month_total                                                  
19128000276 6             1         3        1      -1.0    0        0   
            7             1         3        2      -1.0    0        0   
            8             1         3        3      -1.0    0        0   
            9             1         3        4      -1.0    0        0   
            10            1         3        1      -1.0    0        0   

                         erbatamt  erbatamt_lag time_interact  \
SSUID       month_total                                         
19128000276 6                 0.0           0.0      hh-6-3-1   
            7                 0.0           0.0      hh-7-3-1   
            8                 0.0           0.0      hh-8-3-1   
            9                 0.0           0.0      hh-9-3-1   
            10                0.0           0.0     hh-10-3-1   

                         erbatamt_fitted_base_reb  \
SSUID       month_total                             
19128000276 6                            1.689446   
            7                            1.689446   
            8                            1.689446   
            9                            1.689446   
            10                           1.689446   

                         erbatamt_fitted_base_reb_lag  const  reb_avg_amt  \
SSUID       month_total                                                     
19128000276 6                                2.478895      1          0.0   
            7                                2.478895      1          0.0   
            8                                2.478895      1          0.0   
            9                                2.478895      1          0.0   
            10                               2.478895      1          0.0   

                         reb_lag_avg_amt  
SSUID       month_total                   
19128000276 6                        0.0  
            7                        0.0  
            8                        0.0  
            9                        0.0  
            10                       0.0

In [250]:
reb_amt_avg_ems0 = round(hh_gr2[(hh_gr2['ERBATAMT']>0) & (hh_gr2['ems'] == '0')]['ERBATAMT'].mean(),2) # 665.80
reb_amt_avg_ems1 = round(hh_gr2[(hh_gr2['ERBATAMT']>0) & (hh_gr2['ems'] == '1')]['ERBATAMT'].mean(),2) # 1152.42
print('avg0: ', reb_amt_avg_ems0, '\navg1: ', reb_amt_avg_ems1)

avg0:  665.8 
avg1:  1152.42


In [294]:
# each group needs their own rebate dummy column so that the amount columns are 0 if HH is rebated but not in the group
hh_gr2[['reb_ems0', 'reb_lag_ems0', 'reb_ems1', 'reb_lag_ems1']] = 0

hh_gr2.loc[(hh_gr2.ems == '0') & (hh_gr2.reb == 1), ['reb_ems0']] = 1
hh_gr2.loc[(hh_gr2.ems == '0') & (hh_gr2.reb_lag == 1), ['reb_lag_ems0']] = 1

hh_gr2.loc[(hh_gr2.ems == '1') & (hh_gr2.reb == 1), ['reb_ems1']] = 1
hh_gr2.loc[(hh_gr2.ems == '1') & (hh_gr2.reb_lag == 1), ['reb_lag_ems1']] = 1

# each group has their own rebate amount columns to get estimators for each group
hh_gr2['reb_avg_amt_ems0'] = hh_gr2['reb_ems0']*reb_amt_avg_ems0
hh_gr2['reb_lag_avg_amt_ems0'] = hh_gr2['reb_lag_ems0']*reb_amt_avg_ems0

hh_gr2['reb_avg_amt_ems1'] = hh_gr2['reb_ems1']*reb_amt_avg_ems1
hh_gr2['reb_lag_avg_amt_ems1'] = hh_gr2['reb_lag_ems1']*reb_amt_avg_ems1

hh_gr2[['reb_ems0', 'reb_lag_ems0', 'reb_ems1', 'reb_lag_ems1', 'tfearn']].describe()

,reb_ems0,reb_lag_ems0,reb_ems1,reb_lag_ems1,tfearn
count,196864.000000,196864.000000,196864.000000,196864.000000,196864.000000
mean,0.046215,0.069358,0.067427,0.102675,5317.208540
std,0.209950,0.254062,0.250761,0.303535,5807.721628
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1670.000000
50%,0.000000,0.000000,0.000000,0.000000,4000.000000
75%,0.000000,0.000000,0.000000,0.000000,7181.000000
max,1.000000,1.000000,1.000000,1.000000,98083.000000


In [272]:
exog_vars = hh_gr2[['reb_avg_amt_ems0', 'reb_lag_avg_amt_ems0', 'reb_avg_amt_ems1', 'reb_lag_avg_amt_ems1']]
exog = sm.add_constant(exog_vars)
exog
model_ems = PanelOLS(hh_gr2['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_ems = model_ems.fit()
params_ems

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,tfearn,R-squared:,2.77e-05
Estimator:,PanelOLS,R-squared (Between):,0.0008
No. Observations:,196864,R-squared (Within):,-1.927e-05
Date:,"Thu, Jun 16 2022",R-squared (Overall):,0.0007
Time:,13:33:01,Log-likelihood,-1.776e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.1929
Entities:,24608,P-value,0.3115
Avg Obs:,8.0000,Distribution:,"F(4,172242)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,1.1929


## Grouping 2: Familysize
groups: 1,2,3,4,5,6+

In [301]:
hh_gr3 = hh_new
hh_gr3['month_total'] = hh_gr3['month_total'].astype(float).astype(int)
hh_gr3.set_index(['SSUID', 'month_total'], inplace=True, drop=False)
hh_gr3.head()

ERBATAMT  ERBAMTH  tfearn  month_total        SSUID  \
SSUID       month_total                                                        
19128000276 6                 0.0     -1.0       0            6  19128000276   
            7                 0.0     -1.0       0            7  19128000276   
            8                 0.0     -1.0       0            8  19128000276   
            9                 0.0     -1.0       0            9  19128000276   
            10                0.0     -1.0       0           10  19128000276   

                        ems  EHHNUMPP  SREFMON  EREBATOC  reb  ...  \
SSUID       month_total                                        ...   
19128000276 6             1         3        1      -1.0    0  ...   
            7             1         3        2      -1.0    0  ...   
            8             1         3        3      -1.0    0  ...   
            9             1         3        4      -1.0    0  ...   
            10            1         3        1      -1.0    0  ...   

                         reb_avg_amt_famsize2  reb_lag_avg_amt_famsize2  \
SSUID       month_total                                                   
19128000276 6                             0.0                       0.0   
            7                             0.0                       0.0   
            8                             0.0                       0.0   
            9                             0.0                       0.0   
            10                            0.0                       0.0   

                         reb_avg_amt_famsize3 reb_lag_avg_amt_famsize3  \
SSUID       month_total                                                  
19128000276 6                             0.0                      NaN   
            7                             0.0                      NaN   
            8                             0.0                      NaN   
            9                             0.0                      NaN   
            10                            0.0                      NaN   

                         reb_avg_amt_famsize4  reb_lag_avg_amt_famsize4  \
SSUID       month_total                                                   
19128000276 6                             0.0                       0.0   
            7                             0.0                       0.0   
            8                             0.0                       0.0   
            9                             0.0                       0.0   
            10                            0.0                       0.0   

                         reb_avg_amt_famsize5  reb_lag_avg_amt_famsize5  \
SSUID       month_total                                                   
19128000276 6                             0.0                       0.0   
            7                             0.0                       0.0   
            8                             0.0                       0.0   
            9                             0.0                       0.0   
            10                            0.0                       0.0   

                         reb_avg_amt_famsize6  reb_lag_avg_amt_famsize6  
SSUID       month_total                                                  
19128000276 6                             0.0                       0.0  
            7                             0.0                       0.0  
            8                             0.0                       0.0  
            9                             0.0                       0.0  
            10                            0.0                       0.0  

[5 rows x 57 columns]

In [302]:
reb_amt_avg_famsize1 = round(hh_gr3[(hh_gr3['ERBATAMT']>0) & (hh_gr3['EHHNUMPP'] == 1)]['ERBATAMT'].mean(),2) # 512.59
reb_amt_avg_famsize2 = round(hh_gr3[(hh_gr3['ERBATAMT']>0) & (hh_gr3['EHHNUMPP'] == 2)]['ERBATAMT'].mean(),2) # 835.73
reb_amt_avg_famsize3 = round(hh_gr3[(hh_gr3['ERBATAMT']>0) & (hh_gr3['EHHNUMPP'] == 3)]['ERBATAMT'].mean(),2) # 1007.55
reb_amt_avg_famsize4 = round(hh_gr3[(hh_gr3['ERBATAMT']>0) & (hh_gr3['EHHNUMPP'] == 4)]['ERBATAMT'].mean(),2) # 1223.76
reb_amt_avg_famsize5 = round(hh_gr3[(hh_gr3['ERBATAMT']>0) & (hh_gr3['EHHNUMPP'] == 5)]['ERBATAMT'].mean(),2) # 1345.93
reb_amt_avg_famsize6 = round(hh_gr3[(hh_gr3['ERBATAMT']>0) & (hh_gr3['EHHNUMPP'] == 6)]['ERBATAMT'].mean(),2) # 1496.94
print(reb_amt_avg_famsize1, '\n', reb_amt_avg_famsize2, '\n', reb_amt_avg_famsize3, '\n', reb_amt_avg_famsize4, '\n', reb_amt_avg_famsize5, '\n', reb_amt_avg_famsize6)

512.59 
 835.73 
 1007.55 
 1223.76 
 1345.93 
 1496.94


In [328]:
# each group needs their own rebate dummy column so that the amount columns are 0 if HH is rebated but not in the group
hh_gr3[['reb_famsize1', 'reb_lag_famsize1','reb_famsize2', 'reb_lag_famsize2','reb_famsize3', 'reb_lag_famsize3',
        'reb_famsize4', 'reb_lag_famsize4','reb_famsize5', 'reb_lag_famsize5','reb_famsize6', 'reb_lag_famsize6']] = 0

hh_gr3.loc[(hh_gr3.EHHNUMPP == 1) & (hh_gr3.reb == 1), ['reb_famsize1']] = 1
hh_gr3.loc[(hh_gr3.EHHNUMPP == 1) & (hh_gr3.reb_lag == 1), ['reb_lag_famsize1']] = 1

hh_gr3.loc[(hh_gr3.EHHNUMPP == 2) & (hh_gr3.reb == 1), ['reb_famsize2']] = 1
hh_gr3.loc[(hh_gr3.EHHNUMPP == 2) & (hh_gr3.reb_lag == 1), ['reb_lag_famsize2']] = 1

hh_gr3.loc[(hh_gr3.EHHNUMPP == 3) & (hh_gr3.reb == 1), ['reb_famsize3']] = 1
hh_gr3.loc[(hh_gr3.EHHNUMPP == 3) & (hh_gr3.reb_lag == 1), ['reb_lag_famsize3']] = 1

hh_gr3.loc[(hh_gr3.EHHNUMPP == 4) & (hh_gr3.reb == 1), ['reb_famsize4']] = 1
hh_gr3.loc[(hh_gr3.EHHNUMPP == 4) & (hh_gr3.reb_lag == 1), ['reb_lag_famsize4']] = 1

hh_gr3.loc[(hh_gr3.EHHNUMPP == 5) & (hh_gr3.reb == 1), ['reb_famsize5']] = 1
hh_gr3.loc[(hh_gr3.EHHNUMPP == 5) & (hh_gr3.reb_lag == 1), ['reb_lag_famsize5']] = 1

hh_gr3.loc[(hh_gr3.EHHNUMPP == 6) & (hh_gr3.reb == 1), ['reb_famsize6']] = 1
hh_gr3.loc[(hh_gr3.EHHNUMPP == 6) & (hh_gr3.reb_lag == 1), ['reb_lag_famsize6']] = 1


# each group has their own rebate amount columns to get estimators for each group
hh_gr3['reb_avg_amt_famsize1'] = hh_gr3['reb_famsize1']*reb_amt_avg_famsize1
hh_gr3['reb_lag_avg_amt_famsize1'] = hh_gr3['reb_lag_famsize1']*reb_amt_avg_famsize1

hh_gr3['reb_avg_amt_famsize2'] = hh_gr3['reb_famsize2']*reb_amt_avg_famsize2
hh_gr3['reb_lag_avg_amt_famsize2'] = hh_gr3['reb_lag_famsize2']*reb_amt_avg_famsize2

hh_gr3['reb_avg_amt_famsize3'] = hh_gr3['reb_famsize3']*reb_amt_avg_famsize3
hh_gr3['reb_lag_avg_amt_famsize3'] = hh_gr3['reb_lag_famsize3']*reb_amt_avg_famsize3

hh_gr3['reb_avg_amt_famsize4'] = hh_gr3['reb_famsize4']*reb_amt_avg_famsize4
hh_gr3['reb_lag_avg_amt_famsize4'] = hh_gr3['reb_lag_famsize4']*reb_amt_avg_famsize4

hh_gr3['reb_avg_amt_famsize5'] = hh_gr3['reb_famsize5']*reb_amt_avg_famsize5
hh_gr3['reb_lag_avg_amt_famsize5'] = hh_gr3['reb_lag_famsize5']*reb_amt_avg_famsize6

hh_gr3['reb_avg_amt_famsize6'] = hh_gr3['reb_famsize6']*reb_amt_avg_famsize6
hh_gr3['reb_lag_avg_amt_famsize6'] = hh_gr3['reb_lag_famsize6']*reb_amt_avg_famsize6

hh_gr3[['reb_famsize1', 'reb_lag_famsize1','reb_famsize2', 'reb_lag_famsize2','reb_famsize3', 'reb_lag_famsize4',
        'reb_famsize4', 'reb_lag_famsize4','reb_famsize5', 'reb_lag_famsize5','reb_famsize6', 'reb_lag_famsize6']].replace(np.NaN, 0.0, inplace=True)

hh_gr3[['reb_lag_avg_amt_famsize1', 'reb_avg_amt_famsize2', 'reb_lag_avg_amt_famsize2', 'reb_avg_amt_famsize3', 'reb_lag_avg_amt_famsize3', 'reb_avg_amt_famsize4', 
        'reb_lag_avg_amt_famsize4', 'reb_avg_amt_famsize5', 'reb_lag_avg_amt_famsize5', 'reb_avg_amt_famsize6', 'reb_lag_avg_amt_famsize6']].replace(np.NaN, 0.0, inplace=True)

In [329]:
exog_vars = hh_gr3[['reb_lag_avg_amt_famsize1', 'reb_avg_amt_famsize2', 'reb_lag_avg_amt_famsize2', 'reb_avg_amt_famsize3', 'reb_lag_avg_amt_famsize3', 
                'reb_avg_amt_famsize4', 'reb_lag_avg_amt_famsize4', 'reb_avg_amt_famsize5', 'reb_lag_avg_amt_famsize5', 'reb_avg_amt_famsize6', 'reb_lag_avg_amt_famsize6']]
exog = sm.add_constant(exog_vars)
exog
model_famsize = PanelOLS(hh_gr3['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_famsize = model_famsize.fit()
params_famsize

Dep. Variable:,tfearn,R-squared:,9.327e-05
Estimator:,PanelOLS,R-squared (Between):,-2.56e-05
No. Observations:,196864,R-squared (Within):,5.12e-05
Date:,"Thu, Jun 16 2022",R-squared (Overall):,-1.644e-05
Time:,15:15:43,Log-likelihood,-1.776e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.4606
Entities:,24608,P-value,0.1387
Avg Obs:,8.0000,Distribution:,"F(11,172235)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,1.4606


## Grouping 3: shutting down extensive margin effects
-> eliminating households with no monthly labor earnings in the analysis such that estimates refer only to intensive margin decisions

- as robustness check?


> does it make sense that the earnings=0 group has a negative impact of stimulus? how can they earn even less? take period 0 and 1 into the tfearn_zero variable?

In [330]:
hh_gr4 = hh_new
hh_gr4['month_total'] = hh_gr4['month_total'].astype(float).astype(int)
hh_gr4.set_index(['SSUID', 'month_total'], inplace=True, drop=False)
hh_gr4.head()

ERBATAMT  ERBAMTH  tfearn  month_total        SSUID  \
SSUID       month_total                                                        
19128000276 6                 0.0     -1.0       0            6  19128000276   
            7                 0.0     -1.0       0            7  19128000276   
            8                 0.0     -1.0       0            8  19128000276   
            9                 0.0     -1.0       0            9  19128000276   
            10                0.0     -1.0       0           10  19128000276   

                        ems  EHHNUMPP  SREFMON  EREBATOC  reb  ...  \
SSUID       month_total                                        ...   
19128000276 6             1         3        1      -1.0    0  ...   
            7             1         3        2      -1.0    0  ...   
            8             1         3        3      -1.0    0  ...   
            9             1         3        4      -1.0    0  ...   
            10            1         3        1      -1.0    0  ...   

                         reb_avg_amt_famsize2  reb_lag_avg_amt_famsize2  \
SSUID       month_total                                                   
19128000276 6                             0.0                       0.0   
            7                             0.0                       0.0   
            8                             0.0                       0.0   
            9                             0.0                       0.0   
            10                            0.0                       0.0   

                         reb_avg_amt_famsize3 reb_lag_avg_amt_famsize3  \
SSUID       month_total                                                  
19128000276 6                             0.0                      0.0   
            7                             0.0                      0.0   
            8                             0.0                      0.0   
            9                             0.0                      0.0   
            10                            0.0                      0.0   

                         reb_avg_amt_famsize4  reb_lag_avg_amt_famsize4  \
SSUID       month_total                                                   
19128000276 6                             0.0                       0.0   
            7                             0.0                       0.0   
            8                             0.0                       0.0   
            9                             0.0                       0.0   
            10                            0.0                       0.0   

                         reb_avg_amt_famsize5  reb_lag_avg_amt_famsize5  \
SSUID       month_total                                                   
19128000276 6                             0.0                       0.0   
            7                             0.0                       0.0   
            8                             0.0                       0.0   
            9                             0.0                       0.0   
            10                            0.0                       0.0   

                         reb_avg_amt_famsize6  reb_lag_avg_amt_famsize6  
SSUID       month_total                                                  
19128000276 6                             0.0                       0.0  
            7                             0.0                       0.0  
            8                             0.0                       0.0  
            9                             0.0                       0.0  
            10                            0.0                       0.0  

[5 rows x 57 columns]

In [350]:
# eigenen dummy kreieren mit ssuids wo & (hh_gr4.ERBAMTH == hh_gr4.month_total) & (hh_gr4.tfearn == 0)
tfearn_zero_hh = hh_gr4[(hh_gr4['tfearn']==0) & (hh_gr4['ERBAMTH']==hh_gr4['month_total'])]['SSUID'].unique()

reb_amt_avg_ext0 = round(hh_gr4[(hh_gr4['ERBATAMT']>0) & (hh_gr4['SSUID'].isin(tfearn_zero_hh))]['ERBATAMT'].mean(),2) # 648.18
reb_amt_avg_ext1 = round(hh_gr4[(hh_gr4['ERBATAMT']>0) & (~hh_gr4['SSUID'].isin(tfearn_zero_hh))]['ERBATAMT'].mean(),2) # 971.33
print('avg0: ', reb_amt_avg_ext0, '\navg1: ', reb_amt_avg_ext1)

avg0:  648.18 
avg1:  971.33


In [371]:
# each group needs their own rebate dummy column so that the amount columns are 0 if HH is rebated but not in the group
hh_gr4[['reb_ext0', 'reb_lag_ext0', 'reb_ext1', 'reb_lag_ext1']] = 0

# generate reb dummies with the new dummy
hh_gr4.loc[(hh_gr4.SSUID.isin(tfearn_zero_hh)) & (hh_gr4.reb == 1),'reb_ext0'] = 1 
hh_gr4.loc[(hh_gr4.SSUID.isin(tfearn_zero_hh)) & (hh_gr4.reb_lag == 1),'reb_lag_ext0'] = 1 

hh_gr4.loc[(~hh_gr4.SSUID.isin(tfearn_zero_hh)) & (hh_gr4.reb == 1),'reb_ext1'] = 1 
hh_gr4.loc[(~hh_gr4.SSUID.isin(tfearn_zero_hh)) & (hh_gr4.reb_lag == 1),'reb_lag_ext1'] = 1 

# each group has their own rebate amount columns to get estimators for each group
hh_gr4['reb_avg_amt_ext0'] = hh_gr4['reb_ext0']*reb_amt_avg_ext0
hh_gr4['reb_lag_avg_amt_ext0'] = hh_gr4['reb_lag_ext0']*reb_amt_avg_ext0

hh_gr4['reb_avg_amt_ext1'] = hh_gr4['reb_ext1']*reb_amt_avg_ext1
hh_gr4['reb_lag_avg_amt_ext1'] = hh_gr4['reb_lag_ext1']*reb_amt_avg_ext1

hh_gr4[['reb_ext0', 'reb_lag_ext0', 'reb_ext1', 'reb_lag_ext1', 'tfearn']].describe()

,reb_ext0,reb_lag_ext0,reb_ext1,reb_lag_ext1,tfearn
count,196864.000000,196864.000000,196864.000000,196864.000000,196864.000000
mean,0.009728,0.008447,0.103914,0.163585,5317.208540
std,0.098148,0.091521,0.305150,0.369900,5807.721628
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1670.000000
50%,0.000000,0.000000,0.000000,0.000000,4000.000000
75%,0.000000,0.000000,0.000000,0.000000,7181.000000
max,1.000000,1.000000,1.000000,1.000000,98083.000000


In [372]:
exog_vars = hh_gr4[['reb_avg_amt_ext0', 'reb_lag_avg_amt_ext0', 'reb_avg_amt_ext1', 'reb_lag_avg_amt_ext1']]
exog = sm.add_constant(exog_vars)
exog
model_ext = PanelOLS(hh_gr4['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_ext = model_ext.fit()
params_ext

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,tfearn,R-squared:,0.0009
Estimator:,PanelOLS,R-squared (Between):,0.0032
No. Observations:,196864,R-squared (Within):,0.0008
Date:,"Mon, Jun 20 2022",R-squared (Overall):,0.0029
Time:,14:24:42,Log-likelihood,-1.776e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,37.843
Entities:,24608,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(4,172242)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,37.843


## Grouping 4: Annual salary vs paid by the hour

use EPAYHR1 and EPAYHR2. Think about a smarrt way to combine (4 groups?) or use first job only.

## Grouping 5: Labor Force Status and Work Absences
-> was done by Powell in a 2SLS setting


## Grouping 6: Income Brackets

use same brackets as Parker (2017)